<a href="https://colab.research.google.com/github/Nicolas-Ayala-Collazos/ML/blob/main/preprocessing_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Preprocesamiento de Datos

Este notebook documenta los pasos de preprocesamiento de datos realizados en el conjunto de datos de fallas proporcionado.



In [30]:

import pandas as pd
import numpy as np
# Cargar el conjunto de datos
file_path = 'DATAset.csv'  # Reemplaza con la ruta correcta de tu archivo
data_corrected = pd.read_csv(file_path, delimiter=';')
data = pd.read_csv(file_path, delimiter=';')


<ipython-input-30-032a427253e3>:5: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  data_corrected = pd.read_csv(file_path, delimiter=';')
<ipython-input-30-032a427253e3>:6: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, delimiter=';')


In [ ]:

# Inspección de datos: obtener información sobre los tipos de datos y la presencia de valores faltantes
data_info = data_corrected.info()

# Obtener estadísticas descriptivas para las columnas numéricas
data_description = data_corrected.describe()

data_info, data_description

In [16]:

# Convertir las columnas de fecha al tipo datetime y extraer partes de la fecha
date_columns = ['FECHA_INCIO', 'FECHA_FIN', 'FECHA_INCIO_ETAPA', 'FECHA_FIN_ETAPA']
for col in date_columns:
    data[col] = pd.to_datetime(data[col], errors='coerce')  # Convertir a datetime
    data[col + '_YEAR'] = data[col].dt.year
    data[col + '_MONTH'] = data[col].dt.month
    data[col + '_DAY'] = data[col].dt.day
    data[col + '_HOUR'] = data[col].dt.hour
    data[col + '_MINUTE'] = data[col].dt.minute
    data[col + '_SECOND'] = data[col].dt.second

# Realizar one-hot encoding en las columnas categóricas seleccionadas
categorical_cols = ['CAUSA', 'CAUSA_ENEL_INDICADOR', 'TIPO_INCIDENCIA']
data_encoded = pd.get_dummies(data, columns=categorical_cols)

# Inspección de datos: obtener información sobre los tipos de datos y la presencia de valores faltantes
data1 = data_encoded.info()
data_description = data_encoded.describe()

data1, data_description

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324193 entries, 0 to 324192
Columns: 143 entries, NOMBRE_CTO to TIPO_INCIDENCIA_VACIO
dtypes: datetime64[ns](4), float64(14), int64(21), object(15), uint8(89)
memory usage: 161.1+ MB


(None,
              LATITUD       LONGITUD           ANIO            MES  \
 count  324193.000000  324193.000000  324193.000000  324193.000000   
 mean        4.598946     -74.199536    2018.045553       6.850709   
 std         0.022809       0.017686       2.230508       3.324821   
 min         4.447049     -74.476874    2015.000000       1.000000   
 25%         4.587333     -74.210075    2016.000000       4.000000   
 50%         4.601521     -74.200996    2018.000000       8.000000   
 75%         4.611875     -74.191984    2019.000000      10.000000   
 max         4.733377     -74.037942    2023.000000      12.000000   
 
        DIA_INICIO_INCI       CLIENTES        MINUTOS  MINUTOS_INCIDENCIA  \
 count    324193.000000  324193.000000  324193.000000       324193.000000   
 mean         16.735355      83.901374      41.014841          775.804581   
 std           8.430047      98.835117     179.768736         1357.987669   
 min           1.000000       0.000000       0.000000

In [20]:
# Imputar los valores faltantes en 'MUNICIPIO' basado en la moda de su respectivo 'CIRCUITO_ORI'
data_filled = data_encoded.copy()
data_filled['MUNICIPIO'] = data_filled.groupby('CIRCUITO_ORI')['MUNICIPIO'].apply(lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else x.dropna().iloc[0]))

<ipython-input-20-28d8611dbf0d>:3: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  data_filled['MUNICIPIO'] = data_filled.groupby('CIRCUITO_ORI')['MUNICIPIO'].apply(lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else x.dropna().iloc[0]))


In [23]:

# Definir los umbrales para fallas críticas basados en el percentil 95
saidi_threshold = 0.002153  # Reemplazar con el valor real obtenido
saifi_threshold = 0.0000404  # Reemplazar con el valor real obtenido

# Crear una nueva columna que indica si la falla es crítica (1) o no (0)
data_filled['Falla_Critica'] = ((data_filled['SAIDI'] > saidi_threshold) |
                                 (data_filled['SAIFI'] > saifi_threshold)).astype(int)


In [25]:

# Convertir campos string a categorías nominales (numéricas)
non_date_object_columns = ['NOMBRE_CTO', 'CODIGO_CTO', 'ESTACION', 'TRANSFORMADOR', 'REFERENCIA_INCIDENCIA', 'UNIDAD_REGIONAL_RES', 'ZONA_UNIDAD_REGIONAL_RES', 'NIVEL_TENSION', 'MAYOR_3MIN', 'CIRCUITO', 'CIRCUITO_ORI', 'SUBESTACION', 'SUBESTACION_ORI', 'TRANSFORMADOR (#1)', 'MUNICIPIO']  # Aquí van las columnas que no son de fecha ni ya codificadas

for col in non_date_object_columns:
    data_filled[col] = data_filled[col].astype('category').cat.codes


In [31]:

# Identificar y eliminar campos altamente correlacionados
def identify_highly_correlated(df, threshold):
    corr_matrix = df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    return to_drop

# Aplicar la función para identificar correlaciones mayores al umbral (por ejemplo, 0.95)
highly_correlated_cols = identify_highly_correlated(data_filled, 0.95)

# Eliminar las columnas altamente correlacionadas del conjunto de datos
data_filled.drop(columns=highly_correlated_cols, inplace=True)


<ipython-input-31-966e1e4c9526>:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = df.corr().abs()
<ipython-input-31-966e1e4c9526>:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calculamos la matriz de correlación
corr_matrix = data_filled.corr()

# Generamos una máscara para el triángulo superior
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

# Configuramos la figura de matplotlib
plt.figure(figsize=(15, 10))

# Dibujamos el heatmap con la máscara
sns.heatmap(corr_matrix, mask=mask, center=0, annot=False, fmt=".2f",
            linewidths=.5, cbar_kws={"shrink": .5})

plt.title('Heatmap de Correlación')
plt.show()


In [34]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

# Primero seleccionamos todas las características numéricas no dummy
numerical_features = data_filled.select_dtypes(include=['int64', 'float64']).columns.tolist()
non_dummy_features = [feature for feature in numerical_features if '_dummy' not in feature]

# Ahora aplicamos las tres técnicas de normalización/estandarización

# Min-Max Normalization
min_max_scaler = MinMaxScaler()
data_filled[non_dummy_features] = min_max_scaler.fit_transform(data_encoded[non_dummy_features])

# Standardization (Z-score Normalization)
standard_scaler = StandardScaler()
data_filled[non_dummy_features] = standard_scaler.fit_transform(data_encoded[non_dummy_features])

# Robust Scaling
robust_scaler = RobustScaler()
data_filled[non_dummy_features] = robust_scaler.fit_transform(data_encoded[non_dummy_features])

# Las líneas de código son ejemplos y no deben ejecutarse secuencialmente como tal, ya que sobrescribirían los datos.
# Selecciona uno de los métodos para ejecutar.


In [35]:
data_filled

,NOMBRE_CTO,CODIGO_CTO,ESTACION,LATITUD,LONGITUD,ANIO,MES,DIA_INICIO_INCI,TRANSFORMADOR,REFERENCIA_INCIDENCIA,...,TIPO_INCIDENCIA_AVERIA,TIPO_INCIDENCIA_DESCONEXION MANUAL O MANIOBRAS,TIPO_INCIDENCIA_DESLASTRE DE CARGAS,TIPO_INCIDENCIA_DISPARO DE CABECERA,TIPO_INCIDENCIA_PROTECCIONES INTERMEDIAS,TIPO_INCIDENCIA_SIN AVERIA O DEFECTO NO LOCALIZADO,TIPO_INCIDENCIA_TRABAJOS EN TENSION,TIPO_INCIDENCIA_TRABAJOS PROGRAMADOS,TIPO_INCIDENCIA_VACIO,Falla_Critica
0,106,86,12,0.148407,0.761575,-1.000000,-1.166667,-1.000000,1272,23429,...,0,1,0,0,0,0,0,0,0,0.0
1,20,90,12,0.196096,0.396779,-1.000000,-1.166667,-0.846154,1062,23430,...,0,1,0,0,0,0,0,0,0,1.0
2,68,95,12,-0.769881,-0.917445,-1.000000,-1.166667,-0.846154,227,23431,...,0,1,0,0,0,0,0,0,0,0.0
3,25,71,6,-0.165475,-1.259455,-1.000000,-1.166667,-0.692308,3375,23432,...,1,0,0,0,0,0,0,0,0,0.0
4,25,71,6,-0.165475,-1.259455,-1.000000,-1.166667,-0.692308,205,30285,...,0,0,0,0,1,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324188,78,87,12,-0.895590,0.832485,1.666667,0.333333,1.076923,4961,33727,...,0,0,0,0,0,0,0,1,0,0.0
324189,78,87,12,-0.919297,0.811464,1.666667,0.333333,1.076923,4959,33728,...,0,0,0,0,0,0,0,1,0,0.0
324190,47,91,12,-0.059075,0.502595,1.666667,0.333333,1.153846,1673,33729,...,0,0,0,0,1,0,0,0,0,1.0
324191,78,87,12,-1.045576,0.954831,1.666667,0.333333,1.153846,1037,33730,...,0,0,0,0,1,0,0,0,0,1.0
